# ATP Synthase Model 4
In this notebook, we will study a **mechanisms for proton gradient maintenance**

Possible mechanisms
- making pH gradient by diluting vesicles made in ph7 buffer into ph6 medium, lasts 250 sec, may want more permanent method, experimentally simpler
- Incorporate another protein into membrane that acts as efflux pump, lasts 15 minutes, experimentally more difficult BUT WHERE DOES H+ FROM INSIDE THAT GETS PUMPED OUT COME FROM ??
- use an oil - acidic FC-40 oil was added to oil phase of dsGUV, lasted about 125 seconds
- take an entire system from an organism and purify - for example bacteriorhodopsin or chromatophores (both light-activated)

## Altamura mrna biosynthesis with R. sphaeroides 2020 biorxiv
### scipy.odeint

Equations extracted from the supplementary information: Text S7. <br> Parameters: Table S3

In [7]:
# Import cell (refined)
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import colorcet
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
bokeh.io.output_notebook()

from biocrnpyler import *

Loading BokehJS ...

In [5]:
def rhs(var, t, k_atpsyn, KM_adp, k_rnapol, KM_atp, k_dec):
    
    adp,pi, atp,rna = var
    
    dadp_dt = -k_atpsyn * (adp / (KM_adp + adp))
    
    dpi_dt= dadp_dt
    
    datp_dt = k_atpsyn * (adp / (KM_adp + adp)) - k_rnapol *  (atp / (KM_atp + atp))
    
    drna_dt = k_rnapol * (atp / (KM_atp + atp)) - k_dec * rna
    
    return np.array([dadp_dt,dpi_dt, datp_dt, drna_dt])

In [6]:
# # define parameters
# k_atpsyn = 1
# KM_adp = 0.128
# k_rnapol = 1
# KM_atp = 
# k_dec = 1e-10

# # define initial conditions
# adp = 0.2 # mM
# pi = 10
# atp = 0
# rna = 0


Nevermind
### Let's try using biocrnpyler
---
```
ADP + Pi --> ATP (by ATP Synthase)
ATP + nucleotides --> mRNA + Pi (by T7 RNA Polymerase)

```
---

In [16]:
# dna = Species('dna')
# rna = Species('rna')
atp = Species('atp')
adp = Species('adp')
pi = Species('pi')
nuc = Species('nuc')

k1 = 1
k2 = 1

species = [atp, adp, pi, nuc]

rxn1 = Reaction([adp, pi], [atp], propensity_type = 'massaction', k = k1, k_rev = 1e-5)
rxn2 = Reaction([atp], [nuc, adp, pi], propensity_type = 'massaction', k = k2, k_rev = 1e-5)
#rxn3 = Reaction([atp], [adp, pi], propensity_type = 'massaction', k = k1, k_rev = 1e-5)

reactions = [rxn1, rxn2]

CRN_altamura = ChemicalReactionNetwork(species = species, reactions = reactions)

x0_dict = {'atp':20}

timepoints = np.linspace(0,20,1000)

re = CRN_altamura.simulate_with_bioscrape(timepoints, x0_dict)

In [17]:
palette = colorcet.b_glasbey_category10

p3 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'Nucleotide biosynthesis'
        )

p3.line(timepoints, re['atp'], line_width = 2, color = palette[0], legend_label = 'atp' )
p3.line(timepoints, re['adp'], line_width = 2,color = palette[1], legend_label = 'adp' )
p3.line(timepoints, re['pi'], line_width = 2,color = palette[2], legend_label = 'pi' )
p3.line(timepoints, re['nuc'], line_width = 2,color = palette[3], legend_label = 'nuc' )

bokeh.io.show(p3)

This is what we expect. Was difficult to replicate what Altamura did. We have to read it over again. But first make sure it's worth the effort and do some more thinking.

In [18]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,colorcet,scipy,biocrnpyler,jupyterlab
  

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
colorcet 2.0.2
scipy 1.4.1
biocrnpyler unknown
jupyterlab 1.2.6
